In [19]:
from pyspark.sql import SparkSession, functions as F
import pandas as pd

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("Process Cleaned JSON Data").getOrCreate()

df = spark.read.json("/home/yevhen/Проекти/butsa/json/filtered_players.json")

filtered_players = (
    df.filter(
        (df.primaryPosition == "Gk")  # |
        # (df.primaryPosition == "Lwd")
    )
    .sort(df.rTalent.desc(), df.power.desc())
    .select(
        "id",
        "name",
        "country_name",
        "primaryPosition",
        "secondaryPosition",
        "club",
        "rTalent",
        "power",
        "team_id",
        "transfer",
    )
)

# Виводимо результати
filtered_players.show(59, truncate=False)

# Перетворюємо Spark DataFrame у Pandas DataFrame
pandas_df = filtered_players.toPandas()

# Зберігаємо у файл Excel
output_path = "/home/yevhen/Проекти/butsa/json/filtered_players.xlsx"
# with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
#     pandas_df.to_excel(writer, sheet_name="Cf", index=False)

+------+------------------------+------------------------+---------------+-----------------+---------------------+-------+-----+---------------------------------+
|id    |name                    |country_name            |primaryPosition|secondaryPosition|club                 |rTalent|power|team_id                          |
+------+------------------------+------------------------+---------------+-----------------+---------------------+-------+-----+---------------------------------+
|852384|Кармело Бордаберри      |Парагвай                |Cd             |Dm               |Кильмес              |4.9    |28.47|https://www.butsa.ru/roster/10743|
|853405|Алекс Вискарра          |Боливия                 |Cd             |                 |Кильмес              |4.9    |27.53|https://www.butsa.ru/roster/10743|
|852427|Новак Джокович          |Сербия и Черногория     |Cd             |                 |Капошвар             |4.9    |18.96|https://www.butsa.ru/roster/2000 |
|852881|Владимир Басов

In [35]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("Average rTalent by Club").getOrCreate()

# Завантаження JSON у Spark DataFrame
df = spark.read.json("/home/yevhen/Проекти/butsa/json/filtered_players.json")

# Обчислення середнього rTalent по Club
avg_rTalent_by_club = (
    df.groupBy("club")
    .agg(
        F.avg("rTalent").alias("avg_rTalent"),  # Обчислюємо середнє значення rTalent
        F.avg("power").alias("avg_power"),  # Обчислюємо середнє значення power
        F.count("id").alias("count"),  # Рахуємо кількість гравців у клубі
    )
    .filter(F.col("count") >= 6)  # Фільтруємо клуби з середнім rTalent > 0
    .select("club", "avg_rTalent", "avg_power", "count")
    .orderBy(F.desc("avg_rTalent"))  # Сортуємо за спаданням середнього rTalent
)

# Виводимо результати
avg_rTalent_by_club.show(40, truncate=False)

+--------------+-----------------+------------------+-----+
|club          |avg_rTalent      |avg_power         |count|
+--------------+-----------------+------------------+-----+
|Кильмес       |4.871428571428572|28.98857142857143 |7    |
|Спартак Москва|4.45             |25.269000000000005|10   |
|Аль Барк Тарим|4.427272727272727|28.85272727272728 |11   |
|Уракан Манати |4.418181818181818|24.987272727272728|11   |
|Газ Метан     |4.35             |32.375            |8    |
|ПФК Рось      |4.342857142857143|25.555714285714288|7    |
|Виллем II     |4.286666666666667|22.310000000000002|15   |
|АнтА          |4.266666666666667|21.671111111111113|9    |
|Ницца         |4.17             |18.144            |10   |
+--------------+-----------------+------------------+-----+



In [24]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive


def upload_excel_to_google_drive(
    file_path,
    file_name,
    mime_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
):
    """
    Завантажує Excel-файл у Google Диск як Google Sheet.
    Якщо файл із такою назвою вже існує, він буде перезаписаний.

    :param file_path: Шлях до локального файлу Excel.
    :param file_name: Назва файлу, який буде створено або перезаписано на Google Диску.
    :param mime_type: Тип файлу для завантаження (за замовчуванням Google Sheet).
    """
    # Авторизація через GoogleAuth
    gauth = GoogleAuth()
    gauth.LoadClientConfigFile(
        "/home/yevhen/Проекти/butsa/json/client_secret_935743862394-45jpsgc3uv5g890ivtfj5843hj6p4u65.apps.googleusercontent.com.json"
    )
    gauth.LocalWebserverAuth()  # Відкриває браузер для авторизації
    drive = GoogleDrive(gauth)

    # Пошук файлу з такою ж назвою
    file_list = drive.ListFile(
        {"q": f"title='{file_name}' and trashed=false"}
    ).GetList()

    if file_list:
        # Якщо файл знайдено, оновлюємо його
        file = file_list[0]  # Беремо перший файл із такою назвою
        file.SetContentFile(file_path)
        file.Upload()
        print(f"Файл '{file_name}' успішно оновлено на Google Диску.")
    else:
        # Якщо файл не знайдено, створюємо новий
        file = drive.CreateFile({"title": file_name, "mimeType": mime_type})
        file.SetContentFile(file_path)
        file.Upload()


upload_excel_to_google_drive(
    file_path="/home/yevhen/Проекти/butsa/json/filtered_players.xlsx",
    file_name="Filtered players",
)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=935743862394-45jpsgc3uv5g890ivtfj5843hj6p4u65.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
